In [101]:
%matplotlib inline

from hydra import initialize, compose
from omegaconf import OmegaConf 

import dr_gen.utils.run as ru
import dr_gen.utils.display as dsp

from dr_gen.analyze.run_group import RunGroup
import dr_gen.analyze.result_plotting as rplt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup Config and Generator

In [2]:
with initialize(config_path="../configs/", version_base=None):
    cfg = compose(
        config_name="config.yaml",
        overrides=[
            "paths=mac",
        ]
    )

In [3]:
generator = ru.set_deterministic(cfg.seed)

In [4]:
print(OmegaConf.to_yaml(OmegaConf.to_container(cfg.paths, resolve=True)))

root: /Users/daniellerothermel/drotherm
proj_dir_name: dr_gen
data: /Users/daniellerothermel/drotherm/data
logs: /Users/daniellerothermel/drotherm/logs
my_data: /Users/daniellerothermel/drotherm/data/dr_gen
my_logs: /Users/daniellerothermel/drotherm/logs/dr_gen
run_dir: /Users/daniellerothermel/drotherm/logs/dr_gen/bs500/lr0.1/wd0.0001/s0/2025-04-03/10-22-1743690172
dataset_cache_root: /Users/daniellerothermel/drotherm/data/cifar10/
agg_results: /Users/daniellerothermel/drotherm/data/dr_gen/cifar10/cluster_runs/lr_wd_init_v0



### Load, Disect and Filter Sweep

In [5]:
rg = RunGroup()
rg.load_runs_from_base_dir(cfg.paths.agg_results)

>> 0 / 1288 files failed parsing
>> Updated hpm sweep info


In [6]:
rg.ignore_runs_by_hpms(epochs=180)

>> Ignoring rid: 1287
>> Updated hpm sweep info


In [7]:
print(dsp.make_table(*rg.get_swept_table_data()))

+------+------------+
| Key  |   Values   |
+------+------------+
| Init | pretrained |
|      |   random   |
+------+------------+
|  WD  |  6.3e-05   |
|      |   1e-05    |
|      |  0.00016   |
|      |   0.0001   |
|      |  0.00025   |
|      |   4e-05    |
+------+------------+
|  LR  |    0.16    |
|      |    0.04    |
|      |    0.25    |
|      |    0.2     |
|      |    0.06    |
|      |    0.1     |
|      |    0.01    |
+------+------------+


In [8]:
table = dsp.make_table(*rg.get_hpms_sweep_table())
print(">> Current Sweep, Ready to Analyze:")
dsp.print_table(
    table,
    drop_cols=[],
    sort_cols=['Init', 'LR', 'WD'],
    lr=[0.04, 0.06, 0.1, 0.16, 0.25],
)

>> Current Sweep, Ready to Analyze:
+------------+------+---------+-------+
|    Init    |  LR  |    WD   | Count |
+------------+------+---------+-------+
| pretrained | 0.04 |  0.0001 |   20  |
| pretrained | 0.04 | 0.00016 |   20  |
| pretrained | 0.04 | 0.00025 |   20  |
| pretrained | 0.04 |  4e-05  |   20  |
| pretrained | 0.04 | 6.3e-05 |   20  |
| pretrained | 0.06 |  0.0001 |   20  |
| pretrained | 0.06 | 0.00016 |   20  |
| pretrained | 0.06 | 0.00025 |   20  |
| pretrained | 0.06 |  4e-05  |   20  |
| pretrained | 0.06 | 6.3e-05 |   20  |
| pretrained | 0.1  |  0.0001 |  103  |
| pretrained | 0.1  | 0.00016 |   20  |
| pretrained | 0.1  | 0.00025 |   20  |
| pretrained | 0.1  |  4e-05  |   20  |
| pretrained | 0.1  | 6.3e-05 |   20  |
| pretrained | 0.16 |  0.0001 |   20  |
| pretrained | 0.16 | 0.00016 |   20  |
| pretrained | 0.16 | 0.00025 |   20  |
| pretrained | 0.16 |  4e-05  |   20  |
| pretrained | 0.16 | 6.3e-05 |   20  |
| pretrained | 0.25 |  0.0001 |   20  |
| pr

In [9]:
runs_pre = rg.select_run_data_by_hpms(lr=0.1, wd=1e-4, init="pretrained")
for hpm, rlist in runs_pre.items():
    print(f" - {str(hpm):70} | {len(rlist):,} RIDS")

 - model.weights=DEFAULT optim.lr=0.1 optim.weight_decay=0.0001           | 103 RIDS


In [10]:
runs_rand = rg.select_run_data_by_hpms(**{"optim.lr": 0.1, "optim.weight_decay": 1e-4, "init": "random"})
for hpm, rlist in runs_rand.items():
    print(f" - {str(hpm):70} | {len(rlist):,} RIDS")

 - model.weights=None optim.lr=0.1 optim.weight_decay=0.0001              | 99 RIDS


## Test Result Plotting

In [112]:
hpm_specs_one_each = rplt.make_hpm_specs()

In [167]:
compare_stats_one_each = rplt.one_tn_no_hpm_select_compare_weight_init(
    rg, hpm_specs_one_each, 260, 80, num_bootstraps=1000, split="val",
)

In [168]:
rplt.print_comparative_summary_stats(compare_stats_one_each)

>> :: Per Dist Summary Stats ::

Compare using 1000 bootstraps:
  - [(80, 260) | best: 259] model.weights=DEFAULT optim.lr=0.1 optim.weight_decay=0.0001
  - [(80, 260) | best: 259] model.weights=None optim.lr=0.1 optim.weight_decay=0.0001

point
   n          | 80.00000000 | 80.00000000
   mean       | 85.10723321 | 84.42837757
   median     | 85.21596822 | 84.40068401
   min        | 82.01716959 | 82.93535754
   max        | 87.71292905 | 85.43927757
   variance   | 2.08886392 | 0.21661167
   std        | 1.44241337 | 0.46271884
   sem        | 0.16126672 | 0.05173354
   2.5th      | 82.30920454 | 83.58641876
   25th       | 84.08636828 | 84.15328420
   75th       | 86.23650380 | 84.74547013
   97.5th     | 87.32889941 | 85.30421270
   IQR        | 2.15013552 | 0.59218593
std
   n          | 0.00000000 | 0.00000000
   mean       | 0.16116744 | 0.05119270
   median     | 0.25359058 | 0.06178873
   min        | 0.17520245 | 0.50788965
   max        | 0.22186191 | 0.07101572
   variance 